In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv("E:\\leonhardt\\mayi\\data\\train.csv")

In [2]:
df_train=df_train.loc[df_train['label']!=-1]
df_train = df_train.fillna(0)
train_set = df_train[df_train.loc[:, "date"]<=20171005]
eva_set = df_train[df_train.loc[:, "date"]>20171005]
train_label = train_set.loc[:, "label"]
eva_label = eva_set.loc[:, "label"]
train_set.drop(["id","date", "label"], axis=1, inplace=True)
eva_set.drop(["id","date", "label"], axis=1, inplace=True)

D:\anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [3]:
df_test = pd.read_csv("E:\\leonhardt\\mayi\\data\\test_a.csv")
df_test = df_test.fillna(0)
df_test.drop(["date","id"],axis=1, inplace=True)

In [8]:
from sklearn.metrics import roc_auc_score,roc_curve
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1]

In [11]:
import xgboost as xgb
xgb_train = xgb.DMatrix(train_set, label=train_label)
xgb_vaild = xgb.DMatrix(eva_set, label=eva_label)

In [12]:
params={
'booster':'gbtree',
'objective': 'binary:logistic', #单分类的问题
'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':12, # 构建树的深度，越大越容易过拟合
'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.007, # 如同学习率
'seed':10,
'nthread':7,# cpu 线程数
'eval_metric': 'auc',
'n_jobs': -1
}
plst = list(params.items())
num_rounds = 1300 # 迭代次数
watchlist = [(xgb_train, 'train'),(xgb_vaild, 'val')]

#训练模型并保存
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist, feval = atec_metric,verbose_eval=50)

[0]	train-auc:0.958632	val-auc:0.955429	train-atec_metric:0.534378	val-atec_metric:0.365262
[50]	train-auc:0.983283	val-auc:0.967007	train-atec_metric:0.685746	val-atec_metric:0.485268
[100]	train-auc:0.985095	val-auc:0.968053	train-atec_metric:0.698055	val-atec_metric:0.490559
[150]	train-auc:0.986338	val-auc:0.96914	train-atec_metric:0.714237	val-atec_metric:0.494297
[200]	train-auc:0.987461	val-auc:0.971627	train-atec_metric:0.728129	val-atec_metric:0.497862
[250]	train-auc:0.988311	val-auc:0.972731	train-atec_metric:0.739731	val-atec_metric:0.500143
[300]	train-auc:0.989199	val-auc:0.974243	train-atec_metric:0.753538	val-atec_metric:0.504103
[350]	train-auc:0.990053	val-auc:0.973517	train-atec_metric:0.764882	val-atec_metric:0.506099
[400]	train-auc:0.990971	val-auc:0.974507	train-atec_metric:0.77664	val-atec_metric:0.505655
[450]	train-auc:0.991775	val-auc:0.976031	train-atec_metric:0.787072	val-atec_metric:0.505988
[500]	train-auc:0.992458	val-auc:0.976861	train-atec_metric:0.797

In [13]:
xgb_test = xgb.DMatrix(df_test)
y_preds_xgb = model.predict(xgb_test)

In [14]:
print(y_preds_xgb.max())
print(y_preds_xgb.min())

0.9225767
8.566058e-05


In [15]:
np.savetxt("E:\\leonhardt\\mayi\\mayi\\ipython\\prediction/xgb_fil0_1300.txt", y_preds_xgb, delimiter=",")

In [16]:
params={
'booster':'gbtree',
'objective': 'binary:logistic', #单分类的问题
'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':12, # 构建树的深度，越大越容易过拟合
'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.007, # 如同学习率
'seed':10,
'nthread':7,# cpu 线程数
'eval_metric': 'auc',
'n_jobs': -1
}
plst = list(params.items())
num_rounds = 2000 # 迭代次数
watchlist = [(xgb_train, 'train'),(xgb_vaild, 'val')]

#训练模型并保存
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist, feval = atec_metric,verbose_eval=50)

[0]	train-auc:0.958632	val-auc:0.955429	train-atec_metric:0.534378	val-atec_metric:0.365262
[50]	train-auc:0.983283	val-auc:0.967007	train-atec_metric:0.685746	val-atec_metric:0.485268
[100]	train-auc:0.985095	val-auc:0.968053	train-atec_metric:0.698055	val-atec_metric:0.490559
[150]	train-auc:0.986338	val-auc:0.96914	train-atec_metric:0.714237	val-atec_metric:0.494297
[200]	train-auc:0.987461	val-auc:0.971627	train-atec_metric:0.728129	val-atec_metric:0.497862
[250]	train-auc:0.988311	val-auc:0.972731	train-atec_metric:0.739731	val-atec_metric:0.500143
[300]	train-auc:0.989199	val-auc:0.974243	train-atec_metric:0.753538	val-atec_metric:0.504103
[350]	train-auc:0.990053	val-auc:0.973517	train-atec_metric:0.764882	val-atec_metric:0.506099
[400]	train-auc:0.990971	val-auc:0.974507	train-atec_metric:0.77664	val-atec_metric:0.505655
[450]	train-auc:0.991775	val-auc:0.976031	train-atec_metric:0.787072	val-atec_metric:0.505988
[500]	train-auc:0.992458	val-auc:0.976861	train-atec_metric:0.797

In [17]:
y_preds_xgb2000 = model.predict(xgb_test)

In [18]:
print(y_preds_xgb.max())
print(y_preds_xgb.min())
np.savetxt("E:\\leonhardt\\mayi\\mayi\\ipython\\prediction/xgb_fil0_2000.txt", y_preds_xgb, delimiter=",")

0.9225767
8.566058e-05


In [22]:
df_train = df_train.fillna(0)
train_set = df_train[df_train.loc[:, "date"]<20171005]
eva_set = df_train[df_train.loc[:, "date"]>=20171005]
train_label = train_set.loc[:, "label"]
eva_label = eva_set.loc[:, "label"]
train_set.drop(["id","date", "label"], axis=1, inplace=True)
eva_set.drop(["id","date", "label"], axis=1, inplace=True)
xgb_vaild = xgb.DMatrix(train_set, label=train_label)
xgb_train = xgb.DMatrix(eva_set, label=eva_label)

D:\anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
params={
'booster':'gbtree',
'objective': 'binary:logistic', #单分类的问题
'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':12, # 构建树的深度，越大越容易过拟合
'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.007, # 如同学习率
'seed':10,
'nthread':7,# cpu 线程数
'eval_metric': 'auc',
'n_jobs': -1
}
plst = list(params.items())
num_rounds = 3000 # 迭代次数
watchlist = [(xgb_train, 'train'),(xgb_vaild, 'val')]

#训练模型并保存
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist, feval = atec_metric,verbose_eval=50)

[0]	train-auc:0.957966	val-auc:0.935514	train-atec_metric:0.58813	val-atec_metric:0.398538
[50]	train-auc:0.980101	val-auc:0.961617	train-atec_metric:0.692987	val-atec_metric:0.474106
[100]	train-auc:0.984187	val-auc:0.970274	train-atec_metric:0.70948	val-atec_metric:0.478122
[150]	train-auc:0.985364	val-auc:0.972153	train-atec_metric:0.72287	val-atec_metric:0.482209
[200]	train-auc:0.986562	val-auc:0.973966	train-atec_metric:0.737036	val-atec_metric:0.484252
[250]	train-auc:0.987547	val-auc:0.97454	train-atec_metric:0.74782	val-atec_metric:0.486348
[300]	train-auc:0.988673	val-auc:0.975338	train-atec_metric:0.760978	val-atec_metric:0.489766
[350]	train-auc:0.990032	val-auc:0.977354	train-atec_metric:0.771234	val-atec_metric:0.491756
[400]	train-auc:0.991706	val-auc:0.979491	train-atec_metric:0.780745	val-atec_metric:0.494768
[450]	train-auc:0.992682	val-auc:0.980173	train-atec_metric:0.792661	val-atec_metric:0.494786
[500]	train-auc:0.993782	val-auc:0.98087	train-atec_metric:0.803801	